In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install tensorboardX
! pip install tensorrt

In [ ]:
!pip install cython -U
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
!pip install --quiet --editable .
!pip install --quiet sentencepiece

# Baseline


In [ ]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
  ! MAX_EPOCH=10 \
      LR=1e-05 \
      BATCH_SIZE=32 \
      CUDA_VISIBLE_DEVICES=0 fairseq-train /content/drive/MyDrive/NLP/anli/R1/orig/bin \
          --save-dir /content/drive/MyDrive/NLP/checkpoints/R1/baseline \
          --reset-optimizer  \
          --reset-dataloader  \
          --reset-meters  \
          --best-checkpoint-metric accuracy  \
          --maximize-best-checkpoint-metric  \
          --no-epoch-checkpoints \
          --find-unused-parameters \
          --distributed-world-size 1 \
          --task sentence_prediction  \
          --num-classes 3  \
          --init-token 0  \
          --separator-token 2   \
          --max-positions 512  \
          --shorten-method "truncate"  \
          --arch roberta \
          --dropout 0.1  \
          --attention-dropout 0.1  \
          --weight-decay 0.1  \
          --criterion sentence_prediction  \
          --classification-head-name 'anli' \
          --optimizer adam  \
          --adam-betas '(0.9, 0.98)'  \
          --adam-eps 1e-06  \
          --clip-norm 0.0  \
          --lr-scheduler inverse_sqrt  \
          --lr 1e-05 \
          --fp16  \
          --fp16-init-scale 4  \
          --threshold-loss-scale 1  \
          --fp16-scale-window 128  \
          --batch-size 32  \
          --required-batch-size-multiple 1  \
          --max-tokens 4400 \
          --update-freq 1  \
          --max-update 123873 \
          --max-epoch 5 \
          --seed 100

2023-06-11 11:47:00.347975: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-11 11:47:01 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-06-11 11:47:05 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 100, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 4, 'fp16_scale_window': 128, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': 1.0, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parall

In [ ]:
from fairseq.models.roberta import RobertaModel
roberta = RobertaModel.from_pretrained(
    '/content/drive/MyDrive/NLP/checkpoints/R1/baseline',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='/content/drive/MyDrive/NLP/anli/R1/orig/bin'
)
roberta.eval()

In [ ]:
genre = ['R1','R2','R3']
for g in genre:
  input0 = open('/content/drive/MyDrive/NLP/anli/'+g+'/orig/test.raw.input0', "r")
  input1 = open('/content/drive/MyDrive/NLP/anli/'+g+'/orig/test.raw.input1', "r")
  label = open('/content/drive/MyDrive/NLP/anli/'+g+'/orig/test.raw.label', "r")

  accuracy = 0
  total = 0
  for (x1, x2, y) in zip(input0, input1, label):
    tokens = roberta.encode(x1, x2)
    idx = roberta.predict('anli', tokens).argmax().item()
    dictionary = roberta.task.label_dictionary
    pred = dictionary[idx + dictionary.nspecial]
    total = total + 1
    if  (pred == y.strip()) :
      accuracy = accuracy + 1

  print(g,": ",accuracy)
  print(g,": ",total)

R1 :  332
R1 :  1000
R2 :  333
R2 :  1000
R3 :  396
R3 :  1200


# EDA

In [ ]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
  ! MAX_EPOCH=10 \
      LR=1e-05 \
      BATCH_SIZE=32 \
      CUDA_VISIBLE_DEVICES=0 fairseq-train /content/drive/MyDrive/NLP/anli/R1/orig/bin_eda2 \
          --save-dir /content/drive/MyDrive/NLP/checkpoints/R1/eda \
          --reset-optimizer  \
          --reset-dataloader  \
          --reset-meters  \
          --best-checkpoint-metric accuracy  \
          --maximize-best-checkpoint-metric  \
          --no-epoch-checkpoints \
          --find-unused-parameters \
          --distributed-world-size 1 \
          --task sentence_prediction  \
          --num-classes 3  \
          --init-token 0  \
          --separator-token 2   \
          --max-positions 512  \
          --shorten-method "truncate"  \
          --arch roberta \
          --dropout 0.1  \
          --attention-dropout 0.1  \
          --weight-decay 0.1  \
          --criterion sentence_prediction  \
          --classification-head-name 'anli' \
          --optimizer adam  \
          --adam-betas '(0.9, 0.98)'  \
          --adam-eps 1e-06  \
          --clip-norm 0.0  \
          --lr-scheduler inverse_sqrt  \
          --lr 1e-05 \
          --fp16  \
          --fp16-init-scale 4  \
          --threshold-loss-scale 1  \
          --fp16-scale-window 128  \
          --batch-size 32  \
          --required-batch-size-multiple 1  \
          --max-tokens 4400 \
          --update-freq 1  \
          --max-update 123873 \
          --max-epoch 5 \
          --seed 100

2023-06-11 11:58:52.823986: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-11 11:58:55 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-06-11 11:59:01 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 100, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 4, 'fp16_scale_window': 128, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': 1.0, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parall

In [ ]:
from fairseq.models.roberta import RobertaModel
roberta = RobertaModel.from_pretrained(
    '/content/drive/MyDrive/NLP/checkpoints/R1/eda',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='/content/drive/MyDrive/NLP/anli/R1/orig/bin_eda2'
)
roberta.eval()

In [ ]:
genre = ['R1','R2','R3']
for g in genre:
  input0 = open('/content/drive/MyDrive/NLP/anli/'+g+'/orig/test.raw.input0', "r")
  input1 = open('/content/drive/MyDrive/NLP/anli/'+g+'/orig/test.raw.input1', "r")
  label = open('/content/drive/MyDrive/NLP/anli/'+g+'/orig/test.raw.label', "r")

  accuracy = 0
  total = 0
  for (x1, x2, y) in zip(input0, input1, label):
    tokens = roberta.encode(x1, x2)
    idx = roberta.predict('anli', tokens).argmax().item()
    dictionary = roberta.task.label_dictionary
    pred = dictionary[idx + dictionary.nspecial]
    total = total + 1
    if  (pred == y.strip()) :
      accuracy = accuracy + 1


  print(g,": ",accuracy)
  print(g,": ",total)

R1 :  317
R1 :  1000
R2 :  353
R2 :  1000
R3 :  415
R3 :  1200


# UDA

In [ ]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
  ! MAX_EPOCH=10 \
      LR=1e-05 \
      BATCH_SIZE=32 \
      CUDA_VISIBLE_DEVICES=0 fairseq-train /content/drive/MyDrive/NLP/anli/R1/orig/bin_uda \
          --save-dir /content/drive/MyDrive/NLP/checkpoints/R1/uda \
          --reset-optimizer  \
          --reset-dataloader  \
          --reset-meters  \
          --best-checkpoint-metric accuracy  \
          --maximize-best-checkpoint-metric  \
          --no-epoch-checkpoints \
          --find-unused-parameters \
          --distributed-world-size 1 \
          --task sentence_prediction  \
          --num-classes 3  \
          --init-token 0  \
          --separator-token 2   \
          --max-positions 512  \
          --shorten-method "truncate"  \
          --arch roberta \
          --dropout 0.1  \
          --attention-dropout 0.1  \
          --weight-decay 0.1  \
          --criterion sentence_prediction  \
          --classification-head-name 'anli' \
          --optimizer adam  \
          --adam-betas '(0.9, 0.98)'  \
          --adam-eps 1e-06  \
          --clip-norm 0.0  \
          --lr-scheduler inverse_sqrt  \
          --lr 1e-05 \
          --fp16  \
          --fp16-init-scale 4  \
          --threshold-loss-scale 1  \
          --fp16-scale-window 128  \
          --batch-size 32  \
          --required-batch-size-multiple 1  \
          --max-tokens 4400 \
          --update-freq 1  \
          --max-update 123873 \
          --max-epoch 5 \
          --seed 100

2023-06-11 14:45:13.566431: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-11 14:45:14 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-06-11 14:45:19 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 100, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 4, 'fp16_scale_window': 128, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': 1.0, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parall

In [ ]:
from fairseq.models.roberta import RobertaModel
roberta = RobertaModel.from_pretrained(
    '/content/drive/MyDrive/NLP/checkpoints/R1/uda',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='/content/drive/MyDrive/NLP/anli/R1/orig/bin_uda'
)
roberta.eval()

In [ ]:
genre = ['R1','R2','R3']
for g in genre:
  input0 = open('/content/drive/MyDrive/NLP/anli/'+g+'/orig/test.raw.input0', "r")
  input1 = open('/content/drive/MyDrive/NLP/anli/'+g+'/orig/test.raw.input1', "r")
  label = open('/content/drive/MyDrive/NLP/anli/'+g+'/orig/test.raw.label', "r")

  accuracy = 0
  total = 0
  for (x1, x2, y) in zip(input0, input1, label):
    tokens = roberta.encode(x1, x2)
    idx = roberta.predict('anli', tokens).argmax().item()
    dictionary = roberta.task.label_dictionary
    pred = dictionary[idx + dictionary.nspecial]
    total = total + 1
    if  (pred == y.strip()) :
      accuracy = accuracy + 1


  print(g,": ",accuracy)
  print(g,": ",total)

R1 :  287
R1 :  1000
R2 :  365
R2 :  1000
R3 :  404
R3 :  1200


# SSMBA


In [ ]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
  ! MAX_EPOCH=10 \
      LR=1e-05 \
      BATCH_SIZE=32 \
      CUDA_VISIBLE_DEVICES=0 fairseq-train /content/drive/MyDrive/NLP/anli/R1/orig/bin_ssmba \
          --save-dir /content/drive/MyDrive/NLP/checkpoints/R1/ssmba \
          --reset-optimizer  \
          --reset-dataloader  \
          --reset-meters  \
          --best-checkpoint-metric accuracy  \
          --maximize-best-checkpoint-metric  \
          --no-epoch-checkpoints \
          --find-unused-parameters \
          --distributed-world-size 1 \
          --task sentence_prediction  \
          --num-classes 3  \
          --init-token 0  \
          --separator-token 2   \
          --max-positions 512  \
          --shorten-method "truncate"  \
          --arch roberta \
          --dropout 0.1  \
          --attention-dropout 0.1  \
          --weight-decay 0.1  \
          --criterion sentence_prediction  \
          --classification-head-name 'anli' \
          --optimizer adam  \
          --adam-betas '(0.9, 0.98)'  \
          --adam-eps 1e-06  \
          --clip-norm 0.0  \
          --lr-scheduler inverse_sqrt  \
          --lr 1e-05 \
          --fp16  \
          --fp16-init-scale 4  \
          --threshold-loss-scale 1  \
          --fp16-scale-window 128  \
          --batch-size 32  \
          --required-batch-size-multiple 1  \
          --max-tokens 4400 \
          --update-freq 1  \
          --max-update 123873 \
          --max-epoch 5 \
          --seed 100

2023-06-11 15:29:03.384399: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-11 15:29:07 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-06-11 15:29:11 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 100, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 4, 'fp16_scale_window': 128, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': 1.0, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parall

In [ ]:
from fairseq.models.roberta import RobertaModel
roberta = RobertaModel.from_pretrained(
    '/content/drive/MyDrive/NLP/checkpoints/R1/ssmba',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='/content/drive/MyDrive/NLP/anli/R1/orig/bin_ssmba'
)
roberta.eval()

In [ ]:
genre = ['R1','R2','R3']
for g in genre:
  input0 = open('/content/drive/MyDrive/NLP/anli/'+g+'/orig/test.raw.input0', "r")
  input1 = open('/content/drive/MyDrive/NLP/anli/'+g+'/orig/test.raw.input1', "r")
  label = open('/content/drive/MyDrive/NLP/anli/'+g+'/orig/test.raw.label', "r")

  accuracy = 0
  total = 0
  for (x1, x2, y) in zip(input0, input1, label):
    tokens = roberta.encode(x1, x2)
    idx = roberta.predict('anli', tokens).argmax().item()
    dictionary = roberta.task.label_dictionary
    pred = dictionary[idx + dictionary.nspecial]
    total = total + 1
    if  (pred == y.strip()) :
      accuracy = accuracy + 1


  print(g,": ",accuracy)
  print(g,": ",total)

R1 :  331
R1 :  1000
R2 :  360
R2 :  1000
R3 :  401
R3 :  1200


# EDA_extended

In [ ]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
  ! MAX_EPOCH=10 \
      LR=1e-05 \
      BATCH_SIZE=32 \
      CUDA_VISIBLE_DEVICES=0 fairseq-train /content/drive/MyDrive/NLP/anli/R1/orig/bin_eda_extended \
          --save-dir /content/drive/MyDrive/NLP/checkpoints/R1/eda_extended \
          --reset-optimizer  \
          --reset-dataloader  \
          --reset-meters  \
          --best-checkpoint-metric accuracy  \
          --maximize-best-checkpoint-metric  \
          --no-epoch-checkpoints \
          --find-unused-parameters \
          --distributed-world-size 1 \
          --task sentence_prediction  \
          --num-classes 3  \
          --init-token 0  \
          --separator-token 2   \
          --max-positions 512  \
          --shorten-method "truncate"  \
          --arch roberta \
          --dropout 0.1  \
          --attention-dropout 0.1  \
          --weight-decay 0.1  \
          --criterion sentence_prediction  \
          --classification-head-name 'anli' \
          --optimizer adam  \
          --adam-betas '(0.9, 0.98)'  \
          --adam-eps 1e-06  \
          --clip-norm 0.0  \
          --lr-scheduler inverse_sqrt  \
          --lr 1e-05 \
          --fp16  \
          --fp16-init-scale 4  \
          --threshold-loss-scale 1  \
          --fp16-scale-window 128  \
          --batch-size 32  \
          --required-batch-size-multiple 1  \
          --max-tokens 4400 \
          --update-freq 1  \
          --max-update 123873 \
          --max-epoch 5 \
          --seed 100

2023-06-15 10:24:46.745112: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-15 10:24:48 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-06-15 10:24:51 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 100, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 4, 'fp16_scale_window': 128, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': 1.0, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parall

In [ ]:
from fairseq.models.roberta import RobertaModel
roberta = RobertaModel.from_pretrained(
    '/content/drive/MyDrive/NLP/checkpoints/R1/eda_extended',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='/content/drive/MyDrive/NLP/anli/R1/orig/bin_eda_extended'
)
roberta.eval()

In [ ]:
genre = ['R1','R2','R3']
for g in genre:
  input0 = open('/content/drive/MyDrive/NLP/anli/'+g+'/orig/test.raw.input0', "r")
  input1 = open('/content/drive/MyDrive/NLP/anli/'+g+'/orig/test.raw.input1', "r")
  label = open('/content/drive/MyDrive/NLP/anli/'+g+'/orig/test.raw.label', "r")

  accuracy = 0
  total = 0
  for (x1, x2, y) in zip(input0, input1, label):
    tokens = roberta.encode(x1, x2)
    idx = roberta.predict('anli', tokens).argmax().item()
    dictionary = roberta.task.label_dictionary
    pred = dictionary[idx + dictionary.nspecial]
    total = total + 1
    if  (pred == y.strip()) :
      accuracy = accuracy + 1


  print(g,": ",accuracy)
  print(g,": ",total)

R1 :  340
R1 :  1000
R2 :  358
R2 :  1000
R3 :  422
R3 :  1200
